<a href="https://colab.research.google.com/github/miriammazzeo95/BigData_and_Timeseries_in_Pyspark/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Set Up Pyspark, Imports and Functions**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TO CHANGE CURRENT DIR
# %cd /content/drive/MyDrive/Colab\ Notebooks/Big\ Data\ with\ Pyspark

/content/drive/MyDrive/Colab Notebooks/Big Data with Pyspark


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# https://colab.research.google.com/drive/1YGZCoCGw632dFe_yxAViQYIsXnFKqEdA?usp=sharing
colb_script_id = '1YGZCoCGw632dFe_yxAViQYIsXnFKqEdA'
link_to_file_in_drive = drive.CreateFile({'id':colb_script_id})

link_to_file_in_drive.GetContentFile('Pyspark_ConfigurationImportsFunctions.ipynb') # creates a local copy in the VM
!jupyter nbconvert --to python 'Pyspark_ConfigurationImportsFunctions.ipynb' # converts the local copy from notebook to .py
!rm Pyspark_ConfigurationImportsFunctions.ipynb # deletes the local copy
import Pyspark_ConfigurationImportsFunctions as pyspark_config # imports everything from the script
dir(pyspark_config)

[NbConvertApp] Converting notebook Pyspark_ConfigurationImportsFunctions.ipynb to python
[NbConvertApp] Writing 13668 bytes to Pyspark_ConfigurationImportsFunctions.py
/
start file


['ArrayType',
 'BinaryType',
 'BooleanType',
 'ByteType',
 'DataFrame',
 'DataType',
 'DateType',
 'DecimalType',
 'DoubleType',
 'FloatType',
 'IntegerType',
 'LongType',
 'MAXYEAR',
 'MINYEAR',
 'MapType',
 'NullType',
 'PandasUDFType',
 'ShortType',
 'SparkSession',
 'StringType',
 'StructField',
 'StructType',
 'TimestampType',
 'UserDefinedFunction',
 'WRAPPER_ASSIGNMENTS',
 'WRAPPER_UPDATES',
 'Window',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'abs',
 'acos',
 'add_index_column',
 'add_months',
 'approxCountDistinct',
 'approx_count_distinct',
 'array',
 'array_contains',
 'array_distinct',
 'array_except',
 'array_intersect',
 'array_join',
 'array_max',
 'array_min',
 'array_position',
 'array_remove',
 'array_repeat',
 'array_sort',
 'array_union',
 'arrays_overlap',
 'arrays_zip',
 'asc',
 'asc_nulls_first',
 'asc_nulls_last',
 'ascii',
 'asin',
 'atan',
 'atan2',
 'avg',
 'base64',
 'basestring',
 'bin

# **Ordering Time Series file in folders**

> This script is meant to 'reorder' files to the right monthly and daily folder. The code process parquet data files, collected daily by meters since 4 years. The files are contained in month and day folders, in theese folders files with wrong timestamp are contained as well. The code process files relatively slow to avoid exceeding memory 



In [ ]:
def clean_agg(df, ff=True):
    
    # UNCOMMENT WHENEVER THERE ARE EMPTY PARQUET FILE
    #if df.first() == None:
    #    remove_file(p)
    #    print(f'REMOVED {p} ')
    #    return df
    #else:
    # ...
    
    # DROP COLS
    df=df.drop('MeasuredSpecies', 'MeasuredProperty', 'MeterRegister') 
    if ff:
        # FORWARD FILL (NUMERICAL VALUES)
        df = forward_fill_int(df, 'MeterValue',  ['Measurement','Time-Quarter'], 'MeterValue')
    # AGGREGATION OF MeterValue ON HOUR (keeping same DATETIME, MEASUREMENT AND COMPONENT TYPES, ADDRESS)
    df = df.withColumnRenamed("ProductcomponentType","Component")
    df = df.groupBy('Dato', 'Hour',  'InstallationAdresse', 'Measurement', 'Component').sum()
    df = df.withColumnRenamed("sum(MeterValue)","Value")
    # DROP COLUMN
    df = df.drop(col("Time-Quarter"))
    # ADD DAY COLUMN IF NECESSARY
    #df = df.withColumn('day', dayofmonth('Dato'))
    return df

## SET UP READING PATH FROM HDFS - WHERE DATAFRAMES ARE STORED IN PARQUET
# dataframes are stored in year=../month=../day=.. folders
reading_folder = '/feddl/work/miriam/orderedData'
saving_folder = '/feddl/work/miriam/orderedData'
test_folder = '/feddl/work/miriam/test'

# RETRIEVING ALL PATHS IN THE READING FOLDER - DATAFRAMES
formatting_paths = get_filePaths( f'hdfs dfs -ls {reading_folder}/*/*') # get all the paths to day-folders
formatting_months_paths = get_filePaths( f'hdfs dfs -ls {reading_folder}/*/') # get all the paths to month folders

# TOTAL NUMBER OF FILE PATHS TO BE READ
#num_paths = len(formatting_months_paths)
num_paths = len(formatting_paths)
print(num_paths)

for p in range(1, num_paths):    
    
    # READ FILE  # formatting_months_paths[p] if enough memory
    path = formatting_paths[p] 
    
    print(path)
    df = spark.read.parquet(path)
    
    # DROP UNUSEFUL COLUMNS
    df = df.select([ 'Time-Quarter', 'Dato', 'InstallationAdresse', 'PostNr', 'MeterValue', 'MeasuredSpecies', 'MeterRegister', 'MeasuredProperty', 'ProductcomponentType'])

    # MERGE ADDRESS AND POST CODE
    df = merge_cols(df, 'InstallationAdresse', 'PostNr', 'InstallationAdresse', sep=' ')
    
    # CREATE HOUR COLUMN 
    df = col_to_hour(df, "Time-Quarter", "Hour")
    
    # FILTER OUT ONLY ROWS WHERE ADDRESS IS NOT EMPTY AND NOT ?
    #df = df.where( col('InstallationAdresse').isNotNull() & (col('InstallationAdresse') != '?'))
    
    # ADD COLUMN MEASUREMENT TO FILTER SPECIFIC VALUES MEASURED
    # check column names?
    df = df.withColumn('Measurement', when(col('MeasuredSpecies')=='EL Meter') & (col('MeasuredProperty')=='15minTimeseries-A-') & (col('MeterRegister')=='Active energy A- 15'), 'EL_A_kWh')
        .when(col('MeasuredSpecies')=='EL Meter') & (col('MeasuredProperty') =='15minTimeseries-E-01') & (col('MeterRegister')=='Active energy A+ 15', 'EL_E_kWh' )
        .when(col('MeasuredSpecies')=='EL Meter') & (col('MeasuredProperty') =='Timeseries-Fv-01') & (col('MeterRegister') =='Multienergyunit 1 - value 4 60', 'EL_Fv_m3' )
        .when(col('MeasuredSpecies')=='EL Meter') & (col('MeasuredProperty') =='Timeseries-Va-01') & (col('MeterRegister') =='Multienergyunit 2 - value 1 60', 'EL_Va_m3' )
        .when(col('MeasuredSpecies')=='Watermeter') & (col('MeasuredProperty') =='Timeseries-Fv-01') & (col('MeterRegister') =='Multienergyunit 1 - value 4 60', 'H_M1v4_C' )
        .when(col('MeasuredSpecies')=='Watermeter') & (col('MeasuredProperty') =='Timeseries-Fv-01') & (col('MeterRegister') =='Multienergyunit 3 - value 4 60', 'H_M3v4_m3' )
        .when(col('MeasuredSpecies')=='Watermeter') & (col('MeasuredProperty') =='Timeseries-Va-01') & (col('MeterRegister') =='Multienergyunit 2 - value 2 60', 'W_M2v2_m3' )
        .when(col('MeasuredSpecies')=='Watermeter') & (col('MeasuredProperty') =='Timeseries-Va-01') & (col('MeterRegister') =='Multienergyunit 2 - value 1 60', 'W_M2v1_m3' )
        .when(col('MeasuredSpecies')=='Watermeter') & (col('MeasuredProperty') =='Timeseries-Va-01') & (col('MeterRegister') =='Multienergyunit 2 - value 1 60', 'W_cM2v1_m3' )
        .when(col('MeasuredSpecies')=='Watermeter') & (col('MeasuredProperty') =='Timeseries-Va-01') & (col('MeterRegister') =='Multienergyunit 2 - value 2 60', 'W_cM2v2_m3' )
        .when(col('MeasuredSpecies')=='Power') & (col('MeasuredProperty') =='15minTimeseries-A-') & (col('MeterRegister') =='Active energy A- 15', 'P_A_kWh' )
        .when(col('MeasuredSpecies')=='Power') & (col('MeasuredProperty') =='15minTimeseries-E-01') & (col('MeterRegister') =='Active energy A+ 15', 'P_E_KWh' ))
    
    # FILTER OUT ROWS WITH NULL VALUES IN THE NEW COLUMN Measurement
    # in other words keep only the specified measurement selection above
    df = df.where(df.Measurement.isNotNull())

    # DEBUGGING, IF NEEDED
    #print(df.show(2, truncate=False))
    
    # COLUMNS MERGING/DROPPING, FORWARD FILL INTEGER VALUES, HOURLY AGGREGATION
    df = clean_agg(df, ff=True)
    
    # BUILD PATH TO SAVE CLEANED DATA
    chrono_folder = path.split('g')[-1]
    
    # CREATE SAVING PATH
    saving_path = saving_folder + chrono_folder
    
    # SAVE DAY AS PARQUET FILE
    df.write.parquet(saving_path) # ADD , mode='overwrite' IF NEEDED
    
    # DEBUGGING
    print([p,'SAVED', saving_path])
